In [1]:
import librosa
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np
from tqdm import tqdm

DATA_PATH = "./data/"

def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)


def wav2mfcc(file_path):
    wave, sr = librosa.load(file_path, mono=True, sr=None)

    mfcc = librosa.feature.mfcc(wave, sr=16000)

    return mfcc


def save_data_to_array(path=DATA_PATH):
    labels, _, _ = get_labels(path)

    for label in labels:
      
        mfcc_vectors = []

        wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        for wavfile in tqdm(wavfiles, "Saving vectors of label - '{}'".format(label)):
            mfcc = wav2mfcc(wavfile)
            mfcc_vectors.append(mfcc)
        np.save(label + '.npy', mfcc_vectors)


def get_train_test(split_ratio=0.6, random_state=42):
   
    labels, indices, _ = get_labels(DATA_PATH)

    
    X = np.load(labels[0] + '.npy')
    y = np.zeros(X.shape[0])

    
    for i, label in enumerate(labels[1:]):
        x = np.load(label + '.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))

    assert X.shape[0] == len(y)

    return train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)



def prepare_dataset(path=DATA_PATH):
    labels, _, _ = get_labels(path)
    data = {}
    for label in labels:
        data[label] = {}
        data[label]['path'] = [path  + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]

        vectors = []

        for wavfile in data[label]['path']:
            wave, sr = librosa.load(wavfile, mono=True, sr=None)
 
           
            mfcc = librosa.feature.mfcc(wave, sr=16000)
            vectors.append(mfcc)

        data[label]['mfcc'] = vectors

    return data


def load_dataset(path=DATA_PATH):
    data = prepare_dataset(path)

    dataset = []

    for key in data:
        for mfcc in data[key]['mfcc']:
            dataset.append((key, mfcc))

    return dataset[:100]






D:\p\envs\tensorflow\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
%load_ext autoreload
#自动加载模块
%autoreload 2
#%aimport每次执行键入的Python代码之前，每次重新加载所有模块（排除的除外）。

from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical


feature_dim_2 = 32


save_data_to_array(max_len=feature_dim_2)


X_train, X_test, y_train, y_test = get_train_test()


feature_dim_1 = 20
channel = 1
epochs = 50
batch_size = 100
verbose = 1
num_classes = 3


X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)


Saving vectors of label - 'happy': 100%|██████████████████████████████████████████| 1742/1742 [00:08<00:00, 199.05it/s]


In [3]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model




In [6]:
model = get_model()
model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test_hot))

Train on 3112 samples, validate on 2076 samples
Epoch 1/50
3112/3112 [==============================] - ETA: 33s - loss: 2.0418 - acc: 0.40 - ETA: 10s - loss: 3.5186 - acc: 0.36 - ETA: 6s - loss: 2.8278 - acc: 0.3660 - ETA: 4s - loss: 2.3482 - acc: 0.392 - ETA: 3s - loss: 2.0831 - acc: 0.400 - ETA: 2s - loss: 1.9213 - acc: 0.400 - ETA: 1s - loss: 1.7914 - acc: 0.407 - ETA: 1s - loss: 1.6892 - acc: 0.414 - ETA: 1s - loss: 1.6096 - acc: 0.418 - ETA: 1s - loss: 1.5504 - acc: 0.423 - ETA: 0s - loss: 1.4943 - acc: 0.430 - ETA: 0s - loss: 1.4493 - acc: 0.437 - ETA: 0s - loss: 1.4141 - acc: 0.439 - ETA: 0s - loss: 1.3750 - acc: 0.449 - ETA: 0s - loss: 1.3451 - acc: 0.455 - ETA: 0s - loss: 1.3163 - acc: 0.462 - 2s 743us/step - loss: 1.3152 - acc: 0.4621 - val_loss: 1.1753 - val_acc: 0.3642
Epoch 2/50
3112/3112 [==============================] - ETA: 0s - loss: 1.2440 - acc: 0.420 - ETA: 0s - loss: 0.9809 - acc: 0.546 - ETA: 0s - loss: 0.9190 - acc: 0.596 - ETA: 0s - loss: 0.8915 - acc: 0.612 -

3112/3112 [==============================] - ETA: 0s - loss: 0.0041 - acc: 1.000 - ETA: 0s - loss: 0.0060 - acc: 0.996 - ETA: 0s - loss: 0.0073 - acc: 0.998 - ETA: 0s - loss: 0.0075 - acc: 0.998 - ETA: 0s - loss: 0.0091 - acc: 0.997 - ETA: 0s - loss: 0.0127 - acc: 0.996 - ETA: 0s - loss: 0.0118 - acc: 0.996 - ETA: 0s - loss: 0.0114 - acc: 0.996 - ETA: 0s - loss: 0.0118 - acc: 0.996 - ETA: 0s - loss: 0.0124 - acc: 0.996 - ETA: 0s - loss: 0.0119 - acc: 0.996 - ETA: 0s - loss: 0.0114 - acc: 0.997 - ETA: 0s - loss: 0.0122 - acc: 0.996 - ETA: 0s - loss: 0.0117 - acc: 0.997 - ETA: 0s - loss: 0.0115 - acc: 0.997 - 1s 311us/step - loss: 0.0116 - acc: 0.9971 - val_loss: 0.3504 - val_acc: 0.9355
Epoch 26/50
3112/3112 [==============================] - ETA: 0s - loss: 0.0059 - acc: 1.000 - ETA: 0s - loss: 0.0057 - acc: 1.000 - ETA: 0s - loss: 0.0096 - acc: 0.998 - ETA: 0s - loss: 0.0104 - acc: 0.997 - ETA: 0s - loss: 0.0110 - acc: 0.996 - ETA: 0s - loss: 0.0128 - acc: 0.996 - ETA: 0s - loss: 0.01

3112/3112 [==============================] - ETA: 0s - loss: 3.7052e-04 - acc: 1.000 - ETA: 0s - loss: 0.0015 - acc: 1.0000    - ETA: 0s - loss: 0.0037 - acc: 0.998 - ETA: 0s - loss: 0.0040 - acc: 0.997 - ETA: 0s - loss: 0.0043 - acc: 0.998 - ETA: 0s - loss: 0.0037 - acc: 0.998 - ETA: 0s - loss: 0.0064 - acc: 0.998 - ETA: 0s - loss: 0.0086 - acc: 0.997 - ETA: 0s - loss: 0.0080 - acc: 0.998 - ETA: 0s - loss: 0.0078 - acc: 0.998 - ETA: 0s - loss: 0.0074 - acc: 0.998 - ETA: 0s - loss: 0.0072 - acc: 0.998 - ETA: 0s - loss: 0.0067 - acc: 0.998 - ETA: 0s - loss: 0.0071 - acc: 0.998 - 1s 302us/step - loss: 0.0071 - acc: 0.9984 - val_loss: 0.3339 - val_acc: 0.9566
Epoch 50/50
3112/3112 [==============================] - ETA: 0s - loss: 4.3097e-04 - acc: 1.000 - ETA: 0s - loss: 6.7926e-04 - acc: 1.000 - ETA: 0s - loss: 8.3539e-04 - acc: 1.000 - ETA: 0s - loss: 0.0065 - acc: 0.9986    - ETA: 0s - loss: 0.0096 - acc: 0.995 - ETA: 0s - loss: 0.0087 - acc: 0.996 - ETA: 0s - loss: 0.0118 - acc: 0.99